In [25]:
import glob
import pandas as pd
import numpy as np

In [6]:
fnames = sorted(glob.glob('./data/*'))

In [66]:
df = pd.DataFrame(columns = ['BASIN' , 'CY' , 'YYYYMMDDHH' , 'TECHNUM' , 'TECH' , 'TAU' , 'LatN/S' , 'LonE/W' , 'VMAX' , 'MSLP' ,
'TY' , 'RAD' , 'WINDCODE' , 'RAD1' , 'RAD2' , 'RAD3' , 'RAD4' , 'RADP' , 'RRP' , 'MRD' , 'GUSTS' , 'EYE' ,
'SUBREGION' , 'MAXSEAS' , 'INITIALS' , 'DIR' , 'SPEED' , 'STORMNAME' , 'DEPTH' , 'SEAS' ,
'SEASCODE' , 'SEAS1' , 'SEAS2' , 'SEAS3' , 'SEAS4'])

In [67]:
for f in fnames:
    with open(f) as file:
        lines = [line.rstrip() for line in file]
    for x in lines:
        lst = x.split(', ')[0:35]
        new_row =  lst + [np.nan] * (35 - len(lst))
        df.loc[len(df)] = new_row

In [68]:
df['YYYYMMDDHH'] = pd.to_datetime(df['YYYYMMDDHH'], format = '%Y%m%d%H')
df.set_index('YYYYMMDDHH', inplace = True)
df.sort_index(inplace = True)

In [71]:
df[df['TECH'] == 'BEST']

,BASIN,CY,TECHNUM,TECH,TAU,LatN/S,LonE/W,VMAX,MSLP,TY,...,DIR,SPEED,STORMNAME,DEPTH,SEAS,SEASCODE,SEAS1,SEAS2,SEAS3,SEAS4
YYYYMMDDHH,,,,,,,,,,,,,,,,,,,,,
1982-04-30 06:00:00,IO,01,,BEST,0,118N,824E,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-04-30 12:00:00,IO,01,,BEST,0,125N,821E,25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-04-30 18:00:00,IO,01,,BEST,0,132N,821E,25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-05-01 00:00:00,IO,01,,BEST,0,141N,821E,30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-05-01 06:00:00,IO,01,,BEST,0,147N,824E,30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-05 06:00:00,IO,08,,BEST,0,156N,801E,55,991,TS,...,0,0,MICHAUNG,",",NaN,NaN,NaN,NaN,NaN,NaN
2023-12-05 12:00:00,IO,08,,BEST,0,160N,801E,45,994,TS,...,0,0,MICHAUNG,",",NaN,NaN,NaN,NaN,NaN,NaN
2023-12-05 18:00:00,IO,08,,BEST,0,169N,804E,35,996,TS,...,0,0,MICHAUNG,",",NaN,NaN,NaN,NaN,NaN,NaN
